# Analyse et Exploration des données S3 : Recap et approfondissement

- Année 2025/2026
- Jean Delpech
- Classe : B3 IA/Data (Campus Aix-en-Provence)
- Dernière mise à jour : janvier 2026

In [ ]:
import pandas as pd

## Processus analyse de données / création modèle ML

1. Récupérer les données brutes (.csv, API/JSON, BDD/SQL,…)
2. Parcourir les données, prendre connaissance des colonnes/variables et **réaliser une copie de travail**
3. Statistiques descriptives : moyenne, médiane, max, min, écart-type, quantiles, variance, covariances, corrélation…
4. Nettoyage : gérer les ```NaN``` (suppression ? remplacement par moyenne, médiane ?), convertir les autres valeurs problématiques comme ```inf``` et ```-inf```, supprimmer des colonnes (inutiles, non exploitables)
5. Si besoin, assembler les données (merge), créer les métriques qui nous manquent (créer de nouvelles colonnes, transformer…)
6. Visualiser les données (distribution, mise en relation) avec des graphes adaptés au types de données (bar plots, scatter plots, cat plots, box plots…)
7. Repérer et gérer les outliers (supprimer ou transformer)
8. Choisir, entraîner et tester des modèles de ML ou analyse (inférence statistique)
9. Amélioration des modèles (fine-tuning, feature selection) et comparaison des modèles les mieux ajustés (A/B testing)
10. Déploiement

## Récap : stats descriptives

Les statistiques descriptives permettent de **résumer et caractériser un nuage de points** par quelques valeurs numériques clés. Elles nous renseignent sur la **position**, la **dispersion** et les **relations** entre variables (tendances).

### Mesures de tendance centrale

Ces mesures nous indiquent où se situe le "centre" du nuage de points :

**Moyenne arithmétique** : $\bar{x} = \frac{1}{n}\sum_{i=1}^{n} x_i$

La moyenne représente le centre de gravité du nuage de points. Elle est sensible aux valeurs extrêmes.

**Médiane** : valeur qui sépare l'échantillon en deux parties égales (50% des observations en dessous, 50% au-dessus)

La médiane est plus robuste aux outliers que la moyenne. Elle indique le "milieu" du nuage selon l'axe considéré.

### Mesures d'étendue et de dispersion

Ces mesures quantifient l'**étalement** du nuage de points :

**Minimum et Maximum** : valeurs extrêmes de l'échantillon

Ils définissent la boîte englobante du nuage de points sur chaque axe.

**Variance** : $\sigma^2 = \frac{1}{n}\sum_{i=1}^{n} (x_i - \bar{x})^2$

La variance mesure la dispersion quadratique moyenne autour de la moyenne. Plus elle est élevée, plus le nuage est étalé.

**Écart-type** : $\sigma = \sqrt{\sigma^2}$

L'écart-type s'exprime dans la même unité que les données. Il représente la "largeur" typique du nuage autour de son centre.

**Quantiles** : $Q_p$ est la valeur en dessous de laquelle se trouvent $p\%$ des observations

Les quartiles ($Q_{0.25}$, $Q_{0.5}$, $Q_{0.75}$) permettent de diviser le nuage en quatre parties égales et de visualiser sa distribution via les boîtes à moustaches (*boxplot*, voir plus bas, ou le cours sur la dataviz).

### Mesures de relations entre variables

Pour un nuage de points bivariés (deux variables $X$ et $Y$), ces mesures caractérisent la **forme** et l'**orientation** du nuage :

**Covariance** : $\text{Cov}(X,Y) = \frac{1}{n}\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})$

La covariance indique si le nuage s'étire selon une diagonale montante (covariance positive) ou descendante (covariance négative). Une covariance proche de zéro suggère un nuage sans orientation particulière.

**Coefficient de corrélation de Pearson** : $r = \frac{\text{Cov}(X,Y)}{\sigma_X \sigma_Y}$

Le coefficient $r \in [-1, 1]$ mesure l'intensité de la relation linéaire entre $X$ et $Y$. Un nuage parfaitement aligné selon une droite montante donne $r = 1$, descendante donne $r = -1$, et un nuage sphérique sans relation linéaire donne $r \approx 0$.


Toutes ces statistiques transforment un nuage de points complexe en quelques nombres qui décrivent sa position (moyenne, médiane), son étalement (variance, écart-type, quantiles) et sa forme générale (covariance, corrélation).

Attention, ces statistiques résument des positions/relations complexes en quelques chiffres, plusieurs nuages montrant une structure parfois très différentes peuvent être résumés par les mêmes statistiques. Toujours prendre du recul et conserver son esprit critique.

## Récap : outliers

### Outliers : definition

[Article « Donnée aberrantes sur Wikipédia](https://fr.wikipedia.org/wiki/Donn%C3%A9e_aberrante) :

> « En statistique, une donnée aberrante (anglais outlier) est une valeur ou une observation qui est « distante » des autres observations effectuées sur le même phénomène, c'est-à-dire qu'elle contraste grandement avec les valeurs « normalement » mesurées. Une donnée aberrante peut être due à la variabilité inhérente au phénomène observé, ou indiquer une erreur expérimentale. Dans ce dernier cas, elles sont parfois écartées.

> […]

> Une interprétation statistique naïve d'une série de données contenant des données aberrantes peut être trompeuse et induire en erreur. Par exemple, si une personne décide de calculer la température moyenne de 10 objets dans une pièce, et que 9 d'entre eux ont une température située entre 20 et 25 degrés Celsius mais que le dernier est un four en marche à 175 °C, la médiane de la série sera située entre 20 et 25 °C mais la température moyenne sera entre 35,5 et 40 °C. Dans ce cas, la médiane est un meilleur indicateur de la température des objets que la moyenne. »

![Normal distribution with standard deviation](./images/Empirical_Rule.PNG)

By Dan Kernler - Own work, CC BY-SA 4.0, https://commons.wikimedia.org/w/index.php?curid=36506025

Dans le cas du four donné en exemple ci-dessus, la valeur aberrante est dû au fait qu’elle est produite par une autre « règle » ou un autre « phénomène » que celui qu’on recherche (four allumé vs. chaleur ambiante). C’est ce que l’on pourrait considérer comme une erreur de mesure. 
Une autre catégorie d’outlier sont des valeurs « naturelles », mais très rares (un salaire de 100 k€ par mois, ou un bébé de 7kg à la naissance, ou encore un homme d’une taille de 2,4m – penser à tous les records).

Les outliers peuvent avoir de nombreux effets négatifs :

- influencer négativement l’estimation des distributions et autres patterns sous-jacent aux données (distribution avec une queue plus large que la réalité, moyennes « décalées », etc.)
- variance observée (et erreur) plus importante
- certains modèles (comme la régression traditionnelle) sont très sensibles aux outliers, l’estimation devient moins fiable / stable
- des modèles de ML peuvent faire du surapprentissage sur des données aberrantes, les identifiant comme des données singulièrement importantes et de nouvelles données sans outliers n’étant pas traitées correctement

Les outliers peuvent être de différents types :
- ne concerner qu’une seule variable (dû à une erreur de mesure systématique ou exceptionnelle)
- des combinaisons (patterns) de variables assez rare, mais pour lequel on n’arrive pas à capturer l’élément explicatif (par exemple faible niveau d’étude mais hauts revenus : héritage, footballeur – haut niveau de formation mais pas au sens classique, etc.)
- des mesures effectuées dans des contextes particuliers : on voit une concentration d’outliers pour une location, ou une date donnée qui tranche avec le reste des observation, ici aussi il nous manque un évènement qui a dû survenir pour expliquer le pattern 

La question est : que fait-on des outliers ?

### Repérer les outliers

Une première chose à faire est de les repérer. La boîte à moustache nous permet d’en apprécier visuellement le nombre et la distribution (sont ils rares ? quand il y en a sont-ils très loin de la limite ?). Pour les traiter, nous devons construire des fonctions qui nous permettront de les repérer et de les trier dans nos données. On peut envisager différentes approches :


#### Repérer les outliers à l’aide des IQR

C’est une approche assez universelle, qui fonctionne même pour des distributions légèrement dissymétriques (« à queue épaisse » / *skewed*), et correspond à ce que montre les boîtes à moustache :

In [ ]:
# Répérer les outliers à partir des IQR
def IQR_outliers(df_col: pd.Series) -> tuple:
    Q1 = df_col.quantile(0.25)
    Q3 = df_col.quantile(0.75)
    IQR = Q3 - Q1
    limite_sup = Q3 + 1.5 * IQR
    limite_inf = Q1 - 1.5 * IQR

    return (limite_inf, limite_sup)

Un simple boolean indexing nous permet ensuite d’analyser séparément les données avec ou sans outlier :

```python
sup, inf = IQR_outliers(df.col) 
df_sans_outliers = df[((df.col < sup) & (df.col > inf))]
```

#### Repérer les outliers à partir des écarts-types

On peut fixer une limite en matière d’écart-type (par exemple fixer la limite à 3 écart-types). Cette méthode fonctionne bien pour les distributions normales  :

In [ ]:
def sd_outliers(df_col : pd.Series) -> tuple:
    seuil = 3 # on peut aussi mettre le seuil en argument de la fonction
    limite_sup = df_col.mean() + seuil * df_col.std()
    limite_inf = df_col.mean() - seuil * df_col.std()

    return (limite_inf, limite_sup)

#### Repérer les outliers à partir du z-score

On peut aussi fixer une limite par rapport à la position dans la distribution, grâce au z-score :

In [ ]:
from scipy.stats import zscore
import numpy as np

def z_outliers(df: pd.DataFrame, threshold: float) -> tuple:
    z_scores = np.abs(zscore(df))
    return df[(z_scores < threshold).all(axis=1)]

#### Autres méthodes

Dans certains cas, on peut aller plus loin dans l’élaboration des techniques pour identifier les outliers. Par exemple on peut choisir une distribution de référence (si l’on sait que nos données obéissent à cette loi), et estimer un seuil de probabilité pour cette distribution au delà de laquelle on considère que l’observation est trop improbable pour avoir eu lieu (c’est un peu ce que l’on fait ci-dessus avec des distribution normales). Mais on peut aussi utiliser des algorithmes de machine learning comme le k-mean clustering, et considérer que certaines observations qui sont trop éloignés des clusters qui auront été identifiés sont des outliers. 

### Gérer les outliers

1. Les supprimer simplement et proprement. La taille de l’échantillon est réduite, ce qui peu parfois poser problème.
2. Les « winsoriser » (du nom du bio-statisticien [Charles Winsor](https://en.wikipedia.org/wiki/Charles_Winsor)). Il s’agit de remplacer (et non de supprimer) la valeur des outliers par des seuils (ou des plafonds). Par exemple si dans un ensemble de données qui s’étendent entre 12 (min) et 52 (max) on estime que les valeurs au dessus du 95e percentile sont des outliers, et que ce percentile commence à la valeur 43 par exemple, on va rammener toutes les valeurs au dessus de 43 à 43. Attention le processus doit être symétrique : il faut faire de même avec le 5e percentile, tout en étant vigilant car il faut modifier le même nombre de données de chaque côté de la distribution (ce qui peut n’être pas le cas si on ne considère que les percentiles, quand plusieurs données on la même valeur). La taille de l’échantillon reste intacte.
3. Appliquer des transformations qui « normalisent » les distributions, comme des fonctions logarithmes, des fonctions racines, etc. Mais il faut avoir de bonnes raisons de le faire.
4. Normaliser les données : parfois la normalisation (z-score) ramène l’essentiel des observations dans l’intervalle -3/+3 s.d., et les outliers se retrouvent à une « juste » place où leur influence est atténuée (en valeur absolue)
5. Mettre en œuvre des modèles qui sont réputés plus [robustes](https://en.wikipedia.org/wiki/Robust_regression) à l’influence des outliers que les modèles classiques (les mégthodes d’estimation comme l’OLS y est très sensible).

Dans tous les cas il faut être très prudent dans la gestion des outliers : on modifie des données qui rendent compte de la forme des distribution, notamment les queues, or elles peuvent jouer un rôle important dans les analyses statistiques que l’on veut mener (condition d’application, hypothèses…). C’est pour cela que notre décision dépend aussi du type d’outlier (cf. liste plus haut) qui vont conduire à des actions différentes.

## Récap : Valeurs manquantes

Les valeurs manquantes peuvent avoir les même conséquences que les outliers sur nos analyses (mauvaise estimation de la distribution, mauvaise estimation des paramètres d’un modèle, effets sur la variance, etc.), mais aussi deux autres effets négatifs :

- souvent les valeurs manquantes obéissent à un pattern (recueil de données défectueux, accident, difficultés en un lieu, une date, une cible, en particulier). Si ce pattern n’est pas explicité, il peut contaminer le modèle ou les analyses qui vont mal refléter la réalité (biais statistique)
- s’il manque des données, c’est comme si notre échantillonage n’était pas complet, et donc entame notre capacité à faire ressortir ou identifier dans nos analyses des relations significatives entre les variables (perte de puissance statistique)


#### Type de valeurs manquantes

1. des valeurs qui manquent de manière totalement aléatoire (incident dans un recueil ou la copie d’une donnée)
2. des valeurs qui manquent de manière aléatoire (n’importe quelle valeur peu manquer), mais en fonction de la valeur d’une autre variable, qui elle est connue
3. des valeurs qui manquent de non aléatoire (ce sont juste des valeurs données qui peuvent manquer, par exemple on ne peut pas enregistrer des valeurs supérieur à 10 et cette variable a eu des valeurs supérieures à 10)
4. des valeurs manquante pour des questions (souvent dans les questionnaires), des mesures, ou encore des variables précises au sein des observations
5. des enregistrement entiers qui manquent (pas seulement quelques variables, mais une ligne complète), par exemple un questionnaire rendu vide

On constate qu’encore plus que pour les outliers, le contexte et le type de la valeur manquante vont être très importants pour choisir le traitement adapté. D’ailleurs pour plusieurs cas, à ce niveau du cours, nous n’avons pas les connaissances nécessaires pour mettre en œuvre certaines solutions. On les présente ici juste à titre documentaire. Y revenir quand vous serez mieux outillés en matière de statistiques et de modélisation.

#### Gérer les valeurs manquantes

Comme pour les outliers, il y a deux stratégies : la suppression ou le remplacement.

1. supprimer tout enregistrement (ligne) où une valeur est manquante, avec la méthode ```df.dropna()```. Solution radicale, mais qui risque de diminuer drastiquement la taille de l’échantillon
2. pour les valeurs manquantes totalemenet aléatoires (le cas (1) dans la liste ci-dessus), une bonne technique est de remplacer la valeur manquante par la moyenne ou le mode (la valeur la plus fréquente) de la variable concernée. On appelle ça une imputation. On peut le faire à la main (définir une fonction qui remplace les valeurs manquantes par, au choix, la moyenne ou le mode), mais la bibliothèque de machine learning [Scikit-Learn](https://scikit-learn.org/stable/index.html) dispose de tout un ensemble de méthodes pour cela – module ```sklearn.inpute```. Vous verrez cela dans le module machine learning, pour le moment on fera ces remplacements « à la main ». Cette technique est problématique par contre dans les situations où les valeurs manquantes sont liées à une autre variable (cas (2) et (3) ci-dessus).
3. si on suspecte un pattern derrière les valeurs manquantes, on peut envisager de créer un modèle qui va permettre, à partir des valeurs que prennent les autres variables observées, de recréer les valeurs supposées de la variable qui fait défaut. On peut utiliser un modèle de régression, de corrélation, ou des modèles qui reconstruisent la distribution de la valeur manquante. Des modèles de clustering peuvent aussi intevenir (en repérant d’autres enregistrements qui « ressemblent » à celui où la valeur est absente, mais où elle est présente par contre et où on la recopiera). Il y a d’autres modèles plus complexes adaptés à cet objectif : *multivariate imputation through chained equations, pattern mixture model, predictive mean matching*…

Pour ce cours d’introduction on se limitera aux deux premières solutions.
Quelques conseils : d’abord rechercher si l’absence de valeur obéit à un pattern, la situation la plus favorable étant que ces absences soit totalement aléatoires. Ensuite se demander si la variable affectée est importante pour notre analyse ou pas (inutile de perdre du temps pour rien, avec une mesure de mauvaise qualité). C’est là que la connaissance métier, ou la connaissance du domaine est importante. De la même manière, ne pas négliger la recherche d’outlier, cette inspection des données doit être systématique et effectuée assez tôt dans l’exploration des données.

**Gardez bien une trace de toutes les manipulations que vous effectuez**
Vous aurez vraissemblablement à appliquer différentes méthodes : élimination dans certains, remplacement par des moyennes, idem pour les outliers, etc. Le nettoyage étant lui aussi un processus itératif, il est extrêmement important de rendre vos traitement reproductibles, et traçables (savoir ce que vous avez fait, dans quel ordre…). Donc conserver des logs !

## Méthodes pour l’EDA (dans l’ordre)

In [ ]:
import seaborn as sns

tips_df = sns.load_dataset("tips")
tips_df.shape

Le but premier de l’EDA (*Exploratory Data Analysis*) est de se construire une bonne représentation et surtout compréhension des données en relevant des tendances et en détectant les anomalies. C’est la première étape indispensable de la création d’un modèle de *machine learning* ou de l’analyse de données. C’est à partir de là que vous formulerez les hypothèses (sur les relations entre variables) qui guideront votre travail et que vous allez tester.

Opérationnellement, il s’agit donc d’une part d’établir des statistiques de bases qui pourront nous aider à décrire les données et les tendances que l’on observe, de générer des *dataviz* pertinentes, et d’autre part de préparer les données pour l’analyse à venir, notamment de les nettoyer (valeurs aberrantes, valeurs manquantes).

Une poignée de méthodes sont à connaître absolument car elle nous permettont de réaliser la majeure partie de ce travail à l’aide de la bibliothèque `pandas`, et des bibliothèques de dataviz `seaborn` et `plotly`. Après avoir acquis ainsi une première compréhension des données à l’aide de ces méthodes, vous pourrez achever finement l’EDA en créant des opérations sur mesure pour vos données.

* `df.info()` : la première méthode à appeler une fois que vous aurez chargé vos données sous la forme d’un DataFrame. Cette méthode vous permet d’un coup de disposer des informations suivantes :
  
    * la taille (*shape*) du tableau (lignes / colonnes)
    * le nom de chaque colonne (nom de variables *a priori*)
    * le nombre de valeurs non-nulles dans chaque colonne
    * le datatype des valeurs dans chaque colonne
    * la taille en mémoire du DataFrame 

In [ ]:
tips_df.info()

* pour les données numériques/quantitatives : `df.describe()`. Cette méhode va vous fournir les statistiques descriptives de base (nombre, moyenne, écart-type, médiane, min, max, quartiles…) permettant de se représenter la distribution.

In [ ]:
tips_df.describe()

* Pour les variables catégorielles/qualitatives :  `df['colonne'].value_counts()`. Cette fonction permet de dénombrer rapidement le nombre d’élément dans chaque catégorie (qui sont alors énumérées par la même occasion). Cela remplace avantageusement le recours aux méthoddes `df.unique()̀` et `df.nunique()̀` 

In [ ]:
tips_df['day'].value_counts()

### Dataviz

#### Basique

* `df.plot()`

In [ ]:
tips_df.plot();

* ̀`df['colonne'].hist(bins = nombre_de_barres)` permet d’avoir rapidement une idée de la distribution d’une variable quantitative.

In [ ]:
tips_df['total_bill'].hist(bins=20);

#### Avec `seaborn`

* ̀`sns.histplot()` permet de visualiser une distribution avec en prime l’option `kde` qui permet d’avoir une estimation de la courbe de densité

In [ ]:
sns.histplot(tips_df['total_bill'], kde = True);

#### Avec `plotly`

[plotly](https://plotly.com/python/) est une autre bibliothèque de dataviz qui met l’accent sur l’interactivité (elle forme avec Dash – basé sur Flask – un framework tout à fait adapté au dashboarding). Cette interactivité est intéressante pour l’exploration (possibilité de zoomer, etc.)
Son module `plotly.express` permet de créer des figures complexes en une seule ligne de code, selon un format standardisé :
`nom_de_la_fonction(dataframe,  x= ,  y= ,  title= ,  width= , height= )`, les dimensions étant données en pixel.

[Cheat sheet Plotly](https://media.datacamp.com/legacy/image/upload/v1668605954/Marketing/Blog/Plotly_Cheat_Sheet.pdf)

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

In [ ]:
px.histogram(tips_df, x='total_bill', marginal='box', width=800, height=600)

In [ ]:
from plotly import figure_factory as ff

In [ ]:
group_labels = ['total_bill']
fig = ff.create_distplot([tips_df.total_bill], group_labels)
fig.update_layout(width=800, height=600)

In [ ]:
px.scatter(tips_df, 
           x='total_bill', 
           y='tip', 
           title='Tips x Total_bill', 
           width=600, 
           height=400)

### Valeurs manquantes et aberrantes (outliers)

* `df.count()` pour compter les valeurs non-manquantes (non NA) 

In [ ]:
tips_df.count()

* `df.isnull()` ou `df.isna()` retournent un dataframe booléen où les valeurs `None` ou `np.NaN` sont à `True`. `df.notna()` fait l’opposé. Attention, les chaînes vides `''` ou des valeurs comme `np.inf` ne comptent pas comme des `NA` : il est important de bien inspecter les données pour voir comment identifier les valeurs manquantes (codes ? – chaînes ou valeurs numériques spécifiques, par exemple il n’est pas rare d’avoir des valeurs négatives là où elles ne devraient être que positives, chaînes vides ? etc.)

In [ ]:
tips_na_df = tips_df.copy()
tips_na_df.iloc[tips_na_df.shape[0]-1,0] = np.nan # remplace par une valeur nulle
tips_na_df.isnull()

* `df.dropna()` pour les éliminer. Sinon créer une fonction sur mesure pour remplacer par des valeurs (médiane, mode, etc.) en fonction de la situation  

In [ ]:
tips_na_df.dropna(axis = 0) # attention à l’axe ! axis = 0 par défaut. Attention au inplace = True

* des fonctions pour récupérer les index des valeurs nulles (à modifier pour repérer des valeurs nulles autres que `None` ou `np.nan`) :

In [ ]:
def get_na_index(df):
    return np.where(df.isna())

get_na_index(tips_na_df)

In [ ]:
def get_na_values(df, idx):
    return [df.iloc[i,j] for i,j in zip(*idx)] # on utilise l’opérateur de déballage * pour « dézipper »

get_na_values(tips_na_df, get_na_index(tips_na_df))

* note : voir [cette page](https://www.geeksforgeeks.org/zip-in-python/) si vous n’êtes pas à l’aise avec la fonction `zip()`

* `px.box()` avec `plotly` pour créer des boîtes à moustache où on peut lire facilement les valeurs :

In [ ]:
px.box(tips_df, y="total_bill", width=600, height=400)

* repérer les outliers à l’aide d’une fonction sur mesure (méthode des IQR, voir le récap de la séance 3 pour d’autres méthodes : écart-type, standardisation…)

In [ ]:
# Répérer les outliers à partir des IQR
def IQR_outliers(df_col: pd.Series) -> tuple:
    Q1 = df_col.quantile(0.25)
    Q3 = df_col.quantile(0.75)
    IQR = Q3 - Q1
    limite_sup = Q3 + 1.5 * IQR
    limite_inf = Q1 - 1.5 * IQR

    return (limite_inf, limite_sup)

IQR_outliers(tips_df.total_bill)

### Tendances

* `df.corr()` une table de corrélation nous permet déjà d’émettre des hypothèses sur des relations entre variables quantitatives

In [ ]:
tips_df.columns

In [ ]:
tips_df[['total_bill', 'tip', 'size']].corr()

* `sns.heatmap()` nous permet de visualiser plus facilement les corrélations plt.figure(figsize=(11,7))
sns.heatmap(orders.select_dtypes(exclude = ["object"]).corr(), 
            cmap='coolwarm', 
            annot = True, 
            annot_kws={"size": 12})

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,5))
sns.heatmap(tips_df[['total_bill', 'tip', 'size']].corr(), 
            cmap='coolwarm', 
            annot = True, # affiche les valeurs dans les cellules
            annot_kws={"size": 12}); # taille des chiffres dans les cellulles

* `sns.pairplot()` vous permet de tracer des nuages de points en considérant des variables deux à deux (un peu comme un tableau de corrélation de la dataviz)

In [ ]:
sns.pairplot(tips_df, height=2);

* lorsque l’on a repéré des relations intéressantes, on peut les observer plus en détail avec :
    * `sns.jointplot()` pour avoir une idée des distribution dans un nuage de point
    * `sns.regplot()` pour la relation entre variable quantitative et une estimation de la tendance (droite de regression)
    * `sns.countplot()` avec l’argument `hue=` pour mettre en relation des variables catégorielles
    * `sns.stripplot()` ou `sns.swarmplot()` pour mettre en relation des variables quantitatives et catégorielles, en améliorant la visibilité de la dispersion

In [ ]:
sns.jointplot(x='total_bill', y='tip', data=tips_df);

In [ ]:
sns.regplot(x='total_bill', y='tip', data=tips_df);

In [ ]:
sns.countplot(x='day', data=tips_df, hue='sex');

In [ ]:
sns.stripplot(x='time', y='total_bill', data=tips_df)

* l’argument `trendline` dans un graphe de type `px.scatter()`, permet également d’afficher une tendance avec `plotly`, il faut alors indiquer le type d’estimation voulu. Par exemple, pour une régression un estimateur courant est `ols` (on verra cela en détail dans le cours sur la régression). Cette fonction nécessite la bibliothèque `statsmodels`. `plotly` permet alors d’afficher la valeurs des paramètres estimés et d’autres indicateurs.

In [ ]:
!pip install statsmodels

In [ ]:
px.scatter(tips_df, x='total_bill',y='tip',trendline='ols', width=600, height=400)

### Automatiser l’exploration : `ydata-profiling`

https://pypi.org/project/ydata-profiling/

* créer un rapport d’analyse exploratoire de dataframes, généré automatiquement
* exporter le rapport dans différents formats (HTML, JSON…)
* **Note : ydata est très chatouilleux sur les dépendances, je conseille fortement de mettre en place un environnemnet virtuel dédié, ou d’être très attentifs sur les versions des dépendances (déjà) installées**

Exemple : [EDA du fameux dataset « Titanic »](https://docs.profiling.ydata.ai/latest/examples/titanic/titanic_report.html)

In [ ]:
#!deactivate
#!python3 -m venv ydata
#!source ydata/bin/activate
#!pip install ydata-profiling
from ydata_profiling import ProfileReport
!mkdir reports

In [ ]:
profile = ProfileReport(tips_df, title = 'tips')
profile.to_file('reports/tips_report.html')

## Exercices / Exemples

*Vous n’êtes pas obligés de faire les explorations dans l’ordre : suivez votre inspiration. Mais essayez de tout faire : plus vous explorerez de situations différentes, plus vous serez familier avec ces manipulations.*

Voici quelques jeux de données mis à disposition librement, et qui peuvent servir de support pour mettre en œuvre les techniques d’EDA :

1. Synthétisez les données avec des statistiques descriptives
2. Repérez si ces données doivent être nettoyées (valeurs manquantes ? outliers ?)
3. Repérez des corrélations, regardez si vous pouvez faire émerger des patterns par de simples visualisations
4. N’hésitez pas à simplifier le problème en faisant des hypothèses sur les variables qui vous semblent « à retenir » (lisez bien les métadonnnées si elles sont disponibles)

### Jeu de données 1 : house prices

Le prix de vente de biens immobiliers et de nombreuses variables :
[house prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

Ce jeux de données est proposé dans le cadre d’un sujet de machine learning, nous n’irons pas aussi loin. Nous nous contenterons d’explorer ces données. Aussi seuls trois fichiers nous intéressent : train.csv et test.csv qu’il faudra assembler en un seul dataframe de données, et le fichier texte qui l’accompagne pour bien comprendre les données, ce qui vous permettra de formuler des hypothèses pour orienter votre exploration.



### Jeu de données 2 : loan predication

Voilà un jeu de données intéressant pour étudier la questions des outliers, des transformations de distributions… [loan predication](https://www.kaggle.com/datasets/ninzaami/loan-predication/data)

Le problème est de mettre en relation des caratéristiques d’individus, et la susceptibilité de leur attribuer un prêt.

Réalisez toutes les explorations demandées, mais notez qu’à un moment, selon ce que vous observerez graphiquement, ce jeu de données se prête bien à une transformation par une fonction logarithme ;)

### Jeu de données 3 : pétrole brut et gaz naturel

Ce jeu de données indique la production US de gaz et de pétrole de juin 2008 à juin 2018 : [us-oil-and-gas-production](https://www.kaggle.com/datasets/djzurawski/us-oil-and-gas-production-june-2008-to-june-2018)

Ces données font intervenir des informations temporelles.

En plus des explorations classiques ce jeu de données à pour intérêt :

* il vous faudra convertir des objects (string) en timestamp pour pouvoir exploiter ces données
* avec des ```.groupby()``` essayez de calculer et représenter les productions annuelles de gaz et de pétrole
* de même présentez graphiquement la production par États
* assemblez en un seul dataframe synthétique, par année, la production de gaz **et** de pétrole, pour l’ensemble des US seulement, et un autre, État par État.

### Jeu de données 4 : émissions de CO2

Ce jeu de données répertorie pour des véhicules de différents modèles la quantité de CO2 émise : [vehicle-co2-emissions-dataset](https://www.kaggle.com/datasets/brsahan/vehicle-co2-emissions-dataset/data)

L’occasion de faire quelques visualisation très informatives !

### Jeu de données 5 : des vraies données bien sales

Si vous trouvez les jeux de données précédents trop simples et qu’ils ne demandent pas assez de travail de nettoyage, etc., voici un jeu de données (réelles) qui vous donnera plus de fil à retordre : [Olist dataset](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

Un bon exercice pour apprendre à merger des données, et à créer des métriques.

1. Observez bien le schéma de la base de données
2. Essayez à partir de là de créer un dataframe unique (il va falloir faire des ```.merge(), .groupby()```, etc.) qui contient toutes les données intéressantes, selon des hypothèses que vous pourrez faire : par exemple, y a-t-il une corrélation entre la vitesse à laquelle un consommateur reçoit un produit et l’appréciation qu’il laisse à un vendeur ?

### Présentez un jeu de données qui a attiré votre intérêt pour le projet de ce module.